In [1]:
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem

# Load SDF file
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/special_mol.sdf"
suppl = Chem.SDMolSupplier(sdf_file, removeHs = False)
mol = suppl[0] if suppl else None

if mol:
    mol_block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(mol_block, "mol")  # Add molecule model
    viewer.setStyle({"stick": {}})  # Stick representation
    
    # Add atom labels
    viewer.addLabel("Atom Labels", {"position": {"x": 0, "y": 0, "z": 0}, "backgroundColor": "black", "fontColor": "white"})  # Title label
    for atom in mol.GetAtoms():
        pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
        viewer.addLabel(atom.GetSymbol(), {"position": {"x": pos.x, "y": pos.y, "z": pos.z}, "backgroundColor": "gray", "fontColor": "white"})

    viewer.zoomTo()  # Auto zoom
    viewer.show()

else:
    print("No valid molecules found in SDF file.")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem

# Load SDF file
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/special-rec.sdf"
suppl = Chem.SDMolSupplier(sdf_file, removeHs = False)
mol = suppl[0] if suppl else None

if mol:
    mol_block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(mol_block, "mol")  # Add molecule model
    viewer.setStyle({"stick": {}})  # Stick representation
    
    # Add atom labels
    viewer.addLabel("Atom Labels", {"position": {"x": 0, "y": 0, "z": 0}, "backgroundColor": "black", "fontColor": "white"})  # Title label
    for atom in mol.GetAtoms():
        pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
        viewer.addLabel(atom.GetSymbol(), {"position": {"x": pos.x, "y": pos.y, "z": pos.z}, "backgroundColor": "gray", "fontColor": "white"})

    viewer.zoomTo()  # Auto zoom
    viewer.show()
else:
    print("No valid molecules found in SDF file.")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.